# 10 fold

In [1]:
from PIL import Image
import numpy as np
from reduce_dim import PCA, LDA
from utils_for_data import *
from functions import *
import matplotlib.pyplot as plt
import os
import time
from tqdm.notebook import tqdm

## Make Dataset

In [2]:
data = []
for i in range(1,41):
    path = './att_faces/s'+str(i)
    file_list = os.listdir(path)
    
    for j in range(10):
        with Image.open(path+"/"+file_list[j]) as im:
            a = np.array(im)
            a = a.reshape(112*92,)
            data.append(a)
data = np.asarray(data)

In [3]:
# 데이터 나누기
trunc_data = np.split(data,10,axis=0)

## Set dimensions

In [4]:
dimension_num = 5

## Compute Accuracy 

In [5]:
# 10 fold 정확도 저장
all_pca_acc =[]
all_lda_acc = []

# 10 fold
for i in tqdm(range(10)):
    test = np.array(trunc_data[i])
    
    train = []
    for j in range(10):
        if j!=i:
            train.append(trunc_data[j])
    train = np.array(train)
    train = train.reshape(-1,10304)
    
    # Generate pca, lda model
    pca = PCA(num=dimension_num)
    lda = LDA(num=dimension_num)
    
    # fit 
    pca.fit(train)
    lda.fit(train)
    
    # gallery, query 나누기
    gallery, query = divide(test)
    
    # gallery, query transform
    pca_gallery_transformed = pca.transform(gallery-mean(gallery))
    pca_query_transformed = pca.transform(query-mean(query))
    lda_gallery_transformed = lda.transform(gallery)
    lda_query_transformed = lda.transform(query)
    
    # similarity 계산
    pca_gallery_transformed = pca_gallery_transformed.reshape(28,1,5)
    pca_query_transformed = pca_query_transformed.reshape(1,12,5)
    lda_gallery_transformed = lda_gallery_transformed.reshape(28,1,5)
    lda_query_transformed = lda_query_transformed.reshape(1,12,5)
    
    pca_similarity = np.sum(np.square(pca_query_transformed-pca_gallery_transformed),axis=-1)
    pca_similarity = pca_similarity.T
    
    lda_similarity = np.sum(np.square(lda_query_transformed-lda_gallery_transformed),axis=-1)
    lda_similarity = lda_similarity.T
    
    # acc 계산
    pca_result, pca_acc = compute_result_and_accuracy(pca_similarity)
    all_pca_acc.append(pca_acc)
    
    lda_result, lda_acc = compute_result_and_accuracy(lda_similarity)
    all_lda_acc.append(lda_acc)

  0%|          | 0/10 [00:00<?, ?it/s]

## Average Accuracy

- PCA

In [6]:
sum_ = 0
for i in range(10):
    for j in range(4):
        sum_ += all_pca_acc[i][j][j]
print(sum_/40 )

60.23809523809524


- LDA

In [7]:
sum_ = 0
for i in range(10):
    for j in range(4):
        sum_ += all_lda_acc[i][j][j]
print(sum_/40 )

82.38095238095238
